<a href="https://colab.research.google.com/github/drscook/0m5364_23sp_data_science1/blob/main/materials/m5364_data-science1_notes-day4-numpy-knn.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

#  Numpy & k-Nearest Neighbors
## Class Notes Day 4
## Math 5364 Data Science (masters)
## Tarleton State University
## Dr. Scott Cook

In [1]:
# run this once per session to install packages and helpers
! pip install -q 'git+https://github.com/drscook/helpers'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00


In [2]:
%reload_ext google.colab.data_table
%reload_ext autotime
import helpers.utilities as ut
from helpers.common_imports import *

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                 names=['sepal_length','sepal_width','petal_length','petal_width','class'])#.sample(10)

In [ ]:
import seaborn as sns
sns.pairplot(df, hue='class')

In [ ]:
k = 3  # number of nearest neighbors that we survey to label each row

from scipy.stats import mode
targ = 'class'
feats = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
X = df[feats].values
y = df[targ].values

Y = y[np.newaxis,:]  # insert axis
Z = X[np.newaxis,:] - X[:,np.newaxis]  # insert axes in different places to create Cartesian product
D = np.sum(Z**2, axis=-1)  # square differences in each of the 4 dimension then sum
print('y', y.shape, 'Y', Y.shape, 'X', X.shape, 'Z', Z.shape, 'D', D.shape)
ut.pprint(D)

D[np.diag_indices_from(D)] = np.inf  # prevent a row from counting as its own nearest neighbor
ut.pprint(D)

args = np.argpartition(D, k-1, axis=1)  # args for partial sort of D along each row
ut.pprint(args)

D = np.take_along_axis(D, args, axis=1)  # apply args to D to check that the k smallest distances (nearest neighbors) have sorted to the left
ut.pprint(D)

L = np.take_along_axis(Y, args, axis=1)  # apply args to Y to get labels on those neighbors
L = L[:,:k]  # keep labels of the k nearest neighbors only
ut.pprint(L)

pred = mode(L, axis=1).mode.flatten()  # find most frequent label along each row as class prediction for the row
ut.pprint(pred == y)  # check if prediction is right